This is a very basic demo that really just makes sure you can connect to the Mindlytics session with your apikey and project id.  It is the same code that you can find in examples/basic.  This demo, as well as the other notebook demos, requires that you've set the following environment variables in your shell **before** you launch Jupyter:

```sh
API_KEY="your mindlytics workspace api key"
PROJECT_ID="the project id of a project in that workspace"
```

First, import the Client class from the sdk.

In [1]:
import { Client } from '@mindlytics/node-sdk'

Now we'll define a simple function that gets the currently open session and sends a simple event.

In [2]:
async function toolCall() {
  const session = Client.use()

  const sessionId = session.sessionId

  const userId = session.userId

  console.log(sessionId, userId)

  await session.trackConversationFunction({
    properties: {
      name: "toolCall",
      args: "",
      result: "",
      runtime: 2,
      tool_call_id: '123',
    },
  })
}

Initialize the sdk with your credentials:

In [3]:
const client = new Client({
    projectId: process.env.PROJECT_ID!,
    apiKey: process.env.API_KEY!,
    ...(process.env.BASE_URL && { baseUrl: process.env.BASE_URL }),
    debug: true,
})

Create a new session.

In [4]:
import crypto from 'node:crypto'

const session = client.createSession({
    sessionId: crypto.randomUUID(),
    conversationId: crypto.randomUUID(),
    userId: '123',
})
console.log('Session ID:', session.sessionId)

Session ID: cabe4c93-2f76-4bc3-afb4-aaa2048b3b79


You should see something like `console.log('Session ID:', sessionId) ...` in the console output above, if you've initialed the sdk with debug:true.

Now lets start a context and call our function to send an event.  After that, end the session and flush out the event queue.


In [5]:
await client.withContext(async () => {
    await toolCall()
})

cabe4c93-2f76-4bc3-afb4-aaa2048b3b79 123
[MLSDK:DEBUG:EventQueue] Processing request to /bc/v1/events/event/conversation-function {
  type: 'track',
  event: 'Conversation Function',
  session_id: 'cabe4c93-2f76-4bc3-afb4-aaa2048b3b79',
  user_id: '123',
  device_id: undefined,
  properties: {
    name: 'toolCall',
    args: '',
    result: '',
    runtime: 2,
    tool_call_id: '123'
  },
  conversation_id: 'de8e231f-37cc-4fc5-80a3-2033723d091e'
} {
  header: {
    Authorization: 'e3544a18-7362-4959-a12f-d88ec4abb2df',
    'x-app-id': 'b3003b2b-1c70-42fc-870c-69f76a8e3b2c'
  }
}
[MLSDK:DEBUG] Request: /bc/v1/events/event/conversation-function {
  header: {
    Authorization: 'e3544a18-7362-4959-a12f-d88ec4abb2df',
    'x-app-id': 'b3003b2b-1c70-42fc-870c-69f76a8e3b2c'
  }
}
[MLSDK:DEBUG] Response: /bc/v1/events/event/conversation-function 200 ReadableStream { locked: false, state: 'readable', supportsBYOB: true }


We've sent all the events, now we can end the session and flush the local event queue.

In [6]:
await session.end()
const commErrors = await session.flush()

if (commErrors.length) {
    console.log(`There were ${commErrors.length} communication errors with the Mindlyitcs service!`)
    console.log(commErrors)
}

[MLSDK:DEBUG:EventQueue] Processing request to /bc/v1/events/event/end-session {
  type: 'end_session',
  user_id: '123',
  device_id: undefined,
  session_id: 'cabe4c93-2f76-4bc3-afb4-aaa2048b3b79'
} {
  header: {
    Authorization: 'e3544a18-7362-4959-a12f-d88ec4abb2df',
    'x-app-id': 'b3003b2b-1c70-42fc-870c-69f76a8e3b2c'
  }
}
[MLSDK:DEBUG] Request: /bc/v1/events/event/end-session {
  header: {
    Authorization: 'e3544a18-7362-4959-a12f-d88ec4abb2df',
    'x-app-id': 'b3003b2b-1c70-42fc-870c-69f76a8e3b2c'
  }
}
[MLSDK:DEBUG] Response: /bc/v1/events/event/end-session 200 ReadableStream { locked: false, state: 'readable', supportsBYOB: true }


How do we know if everything worked?  Good question!  If there were any communication errors with the Mindlytics service, the code above would have printed them out.  But the actual analysis of the session was done asynchroniously on the Mindlytics server and normally you would not have access to it here.  You could go to the Mindlytics SaaS portal and look at the conversation in detail there.  For the purposes of this demo, you could use the `dbGET()` method on the client to read the Mindlytics database directly.  Here's how you could fetch the session you just completed along with the events that were generated:


In [7]:
const res = await client.dbGET({collection: 'sessions', op: 'findOne', query: {
    filter: {session_id: session.sessionId},
    include: 'events'
}})

if (res.error) {
    console.log(res.error)
} else {
    for (const ev of res.data.events) {
        console.log(ev.event)
    }
}

[MLSDK:DEBUG] Request: /bc/v1/db/{collection}/{op} {
  query: {
    filter: '{"session_id":"cabe4c93-2f76-4bc3-afb4-aaa2048b3b79"}',
    include: '["events"]'
  },
  path: { collection: 'sessions', op: 'findOne' }
}
[MLSDK:DEBUG] Response: /bc/v1/db/{collection}/{op} 200 ReadableStream { locked: false, state: 'readable', supportsBYOB: true }
Session Started
Conversation Started
Conversation Function
Conversation Ended
Conversation Summary
Session Ended


There is an alternative way to capture these events as they happen in realtime using a websocket connection to the Mindlytics service.  That technique will be demonstrated in other examples.